# Text Mining - Sentimental Analysis - Elon Musk Tweets Data

In [1]:
#Install Lbraries
#!pip install nltk
#!pip install spacy
#!pip install wordcloud
#!pip install -U textblob

### Importing Libraries

In [50]:
import pandas as pd
import numpy as np

import requests
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import twitter_samples

from textblob import TextBlob
from textblob import Word
    
from sklearn.feature_extraction.text import TfidfVectorizer

from bs4 import BeautifulSoup

In [3]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Punith
[nltk_data]     Gowda\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Punith
[nltk_data]     Gowda\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Punith
[nltk_data]     Gowda\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Punith
[nltk_data]     Gowda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Loading Data

In [7]:
data = pd.read_csv('Elon_musk.csv',encoding="latin-1")

In [8]:
df = data.copy()
df.head()

,Unnamed: 0,Text
0,1,@kunalb11 Im an alien
1,2,@ID_AA_Carmack Ray tracing on Cyberpunk with H...
2,3,@joerogan @Spotify Great interview!
3,4,@gtera27 Doge is underestimated
4,5,@teslacn Congratulations Tesla China for amazi...


#### Checking Number of word counts

In [9]:
df['word_count'] = df['Text'].apply(lambda x: len(str(x).split(" ")))
df[['Text','word_count']].head()

,Text,word_count
0,@kunalb11 Im an alien,4
1,@ID_AA_Carmack Ray tracing on Cyberpunk with H...,13
2,@joerogan @Spotify Great interview!,4
3,@gtera27 Doge is underestimated,4
4,@teslacn Congratulations Tesla China for amazi...,17


In [10]:
#droping Unnamed Column
df=df.drop(['Unnamed: 0'],axis=1)

In [11]:
df.head()

,Text,word_count
0,@kunalb11 Im an alien,4
1,@ID_AA_Carmack Ray tracing on Cyberpunk with H...,13
2,@joerogan @Spotify Great interview!,4
3,@gtera27 Doge is underestimated,4
4,@teslacn Congratulations Tesla China for amazi...,17


#### Checking Average Word Length

In [12]:
def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))

df['avg_word'] = data['Text'].apply(lambda x: avg_word(x))
df[['Text','avg_word']].head()

,Text,avg_word
0,@kunalb11 Im an alien,4.750000
1,@ID_AA_Carmack Ray tracing on Cyberpunk with H...,5.384615
2,@joerogan @Spotify Great interview!,8.000000
3,@gtera27 Doge is underestimated,7.000000
4,@teslacn Congratulations Tesla China for amazi...,5.176471


#### Checking Number of stopwords

In [13]:
stop = stopwords.words('english')

df['stopwords']= df['Text'].apply(lambda x: len([x for x in x.split() if x in stop]))
df[['Text','stopwords']].head()

,Text,stopwords
0,@kunalb11 Im an alien,1
1,@ID_AA_Carmack Ray tracing on Cyberpunk with H...,4
2,@joerogan @Spotify Great interview!,0
3,@gtera27 Doge is underestimated,1
4,@teslacn Congratulations Tesla China for amazi...,5


#### Checking Number of special characters

In [14]:
df['hashtags'] = df['Text'].apply(lambda x: len([x for x in x.split() if x.startswith('@')]))
df[['Text','hashtags']].head()

,Text,hashtags
0,@kunalb11 Im an alien,1
1,@ID_AA_Carmack Ray tracing on Cyberpunk with H...,1
2,@joerogan @Spotify Great interview!,2
3,@gtera27 Doge is underestimated,1
4,@teslacn Congratulations Tesla China for amazi...,1


#### Checking number of numerics

In [15]:
df['numerics'] = df['Text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df[['Text','numerics']].head()

,Text,numerics
0,@kunalb11 Im an alien,0
1,@ID_AA_Carmack Ray tracing on Cyberpunk with H...,0
2,@joerogan @Spotify Great interview!,0
3,@gtera27 Doge is underestimated,0
4,@teslacn Congratulations Tesla China for amazi...,0


#### Checking of Upper Case Words

In [16]:
df['upper'] = df['Text'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
df[['Text','upper']].head()

,Text,upper
0,@kunalb11 Im an alien,0
1,@ID_AA_Carmack Ray tracing on Cyberpunk with H...,1
2,@joerogan @Spotify Great interview!,0
3,@gtera27 Doge is underestimated,0
4,@teslacn Congratulations Tesla China for amazi...,0


### Pre - Porcessing

#### Lower case

In [17]:
df['Text'] = df['Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['Text'].head()

0                               @kunalb11 im an alien
1    @id_aa_carmack ray tracing on cyberpunk with h...
2                  @joerogan @spotify great interview!
3                      @gtera27 doge is underestimated
4    @teslacn congratulations tesla china for amazi...
Name: Text, dtype: object

#### Removing Punctution

In [18]:
df['Text'] = df['Text'].str.replace('[^\w\s]','')
df['Text'].head()

0                                 kunalb11 im an alien
1    id_aa_carmack ray tracing on cyberpunk with hd...
2                     joerogan spotify great interview
3                       gtera27 doge is underestimated
4    teslacn congratulations tesla china for amazin...
Name: Text, dtype: object

#### Removing stopwords

In [19]:
stop = stopwords.words('english')
df['Text'] = df['Text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['Text'].head()

0                                    kunalb11 im alien
1    id_aa_carmack ray tracing cyberpunk hdr nextle...
2                     joerogan spotify great interview
3                          gtera27 doge underestimated
4    teslacn congratulations tesla china amazing ex...
Name: Text, dtype: object

#### Removing Common words

In [20]:
freq = pd.Series(' '.join(df['Text']).split()).value_counts()[:10]
freq

spacex            239
amp               218
tesla             166
erdayastronaut    142
rt                127
ppathole          123
flcnhvy           114
yes                86
great              76
teslaownerssv      73
dtype: int64

In [21]:
freq = list(freq.index)
df['Text'] = df['Text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
df['Text'].head()

0                                    kunalb11 im alien
1    id_aa_carmack ray tracing cyberpunk hdr nextle...
2                           joerogan spotify interview
3                          gtera27 doge underestimated
4    teslacn congratulations china amazing executio...
Name: Text, dtype: object

#### Removing of Rare Words

In [22]:
freq = pd.Series(' '.join(df['Text']).split()).value_counts()[-10:]
freq

internships           1
obstacles             1
passes                1
ricguccio             1
sunny                 1
u26a1ufe0f            1
httpstcovrwvop79qd    1
nutshell              1
unveiled              1
httpstco6y42kmcpgk    1
dtype: int64

In [23]:
freq = list(freq.index)
df['Text'] = df['Text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
df['Text'].head()

0                                    kunalb11 im alien
1    id_aa_carmack ray tracing cyberpunk hdr nextle...
2                           joerogan spotify interview
3                          gtera27 doge underestimated
4    teslacn congratulations china amazing executio...
Name: Text, dtype: object

#### Spell Correction

In [24]:
df['Text'][:5].apply(lambda x :str(TextBlob(x).correct()))

0                                    kunalb11 in alien
1    id_aa_carmack ray tracing cyberpunk her nextle...
2                           joerogan specify interview
3                          gtera27 done underestimated
4    teslacn congratulations china amazing executio...
Name: Text, dtype: object

#### Tokenization

In [25]:
TextBlob(df['Text'][1]).words

WordList(['id_aa_carmack', 'ray', 'tracing', 'cyberpunk', 'hdr', 'nextlevel', 'tried'])

In [26]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Punith
[nltk_data]     Gowda\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#### Stemming

In [27]:
from nltk.stem import PorterStemmer


In [28]:
st = PorterStemmer()

df['Text'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))


0                                    kunalb11 im alien
1    id_aa_carmack ray trace cyberpunk hdr nextleve...
2                           joerogan spotifi interview
3                              gtera27 doge underestim
4    teslacn congratul china amaz execut last year ...
Name: Text, dtype: object

#### Lemmatization

In [29]:
from textblob import Word

In [30]:
df["Text"] = df['Text'].apply(lambda x : ' '.join([Word(word).lemmatize() for word in x.split()]))
df['Text'].head()

0                                    kunalb11 im alien
1    id_aa_carmack ray tracing cyberpunk hdr nextle...
2                           joerogan spotify interview
3                          gtera27 doge underestimated
4    teslacn congratulation china amazing execution...
Name: Text, dtype: object

### Advanced Text Processing 

#### N-grams

In [31]:
TextBlob(df['Text'][0]).ngrams(2)

[WordList(['kunalb11', 'im']), WordList(['im', 'alien'])]

#### Teem Frequency

Term frequency is simply the ratio of the count of a word present in a sentence, to the length of the sentence.

Therefore, we can generalize term frequency as:

TF = (Number of times term T appears in the particular row) / (number of terms in that row)

In [32]:
tf1 = (df['Text'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1

,words,tf
0,ray,1
1,id_aa_carmack,1
2,hdr,1
3,tried,1
4,nextlevel,1
5,cyberpunk,1
6,tracing,1


#### Inverse Document Frequency (IDF)

The intuition behind inverse document frequency (IDF) is that a word is not of much use to us if it’s appearing in all the documents.

Therefore, the IDF of each word is the log of the ratio of the total number of rows to the number of rows in which that word is present.

IDF = log(N/n), where, N is the total number of rows and n is the number of rows in which the word was present.

In [33]:
for i,word in enumerate(tf1['words']):
  tf1.loc[i, 'idf'] = np.log(df.shape[0]/(len(df[df['Text'].str.contains(word)])))

tf1

,words,tf,idf
0,ray,1,5.035453
1,id_aa_carmack,1,4.166415
2,hdr,1,6.907255
3,tried,1,5.808643
4,nextlevel,1,6.907255
5,cyberpunk,1,5.115496
6,tracing,1,7.600402


#### Term Frequency – Inverse Document Frequency (TF-IDF)

In [34]:
tf1['tfidf'] = tf1['tf'] * tf1['idf']
tf1

,words,tf,idf,tfidf
0,ray,1,5.035453,5.035453
1,id_aa_carmack,1,4.166415,4.166415
2,hdr,1,6.907255,6.907255
3,tried,1,5.808643,5.808643
4,nextlevel,1,6.907255,6.907255
5,cyberpunk,1,5.115496,5.115496
6,tracing,1,7.600402,7.600402


#### Bag of Words

In [35]:
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
 stop_words= 'english',ngram_range=(1,1))
vect = tfidf.fit_transform(df['Text'])
vect

<1999x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 7373 stored elements in Compressed Sparse Row format>

#### Sentiment Analysis

In [36]:
df['Text'][:5].apply(lambda x: TextBlob(x).sentiment)

0                                 (-0.25, 0.75)
1                                    (0.0, 0.0)
2                                    (0.0, 0.0)
3                                    (0.0, 0.0)
4    (0.20000000000000004, 0.32222222222222224)
Name: Text, dtype: object

In [37]:
df['sentiment'] = df['Text'].apply(lambda x: TextBlob(x).sentiment[0] )
df[['Text','sentiment']].head()

,Text,sentiment
0,kunalb11 im alien,-0.25
1,id_aa_carmack ray tracing cyberpunk hdr nextle...,0.00
2,joerogan spotify interview,0.00
3,gtera27 doge underestimated,0.00
4,teslacn congratulation china amazing execution...,0.20


## Performing Emotion Mining

In [38]:
#import libraries
import codecs
import re
import copy
import collections

from __future__ import division

#### Positive Words

In [39]:
with codecs.open("positive-words.txt", "r", encoding="utf-8") as p:
    pos = p.read()
    print(pos)

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
; 
; Opinion Lexicon: Positive
;
; This file contains a list of POSITIVE opinion words (or sentiment words).
;
; This file and the papers can all be downloaded from 
;    http://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html
;
; If you use this list, please cite one of the following two papers:
;
;   Minqing Hu and Bing Liu. "Mining and Summarizing Customer Reviews." 
;       Proceedings of the ACM SIGKDD International Conference on Knowledge 
;       Discovery and Data Mining (KDD-2004), Aug 22-25, 2004, Seattle, 
;       Washington, USA, 
;   Bing Liu, Minqing Hu and Junsheng Cheng. "Opinion Observer: Analyzing 
;       and Comparing Opinions on the Web." Proceedings of the 14th 
;       International World Wide Web conference (WWW-2005), May 10-14, 
;       2005, Chiba, Japan.
;
; Notes: 
;    1. The appearance of an opinion word in a sentence does not necessarily  
;       mean that the sentence expresse

#### Negative Words

In [40]:
with codecs.open("negative-words.txt", "r", encoding="ISO-8859-1") as n:
    neg = n.read()
    print(neg)

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
; 
; Opinion Lexicon: Negative
;
; This file contains a list of NEGATIVE opinion words (or sentiment words).
;
; This file and the papers can all be downloaded from 
;    http://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html
;
; If you use this list, please cite one of the following two papers:
;
;   Minqing Hu and Bing Liu. "Mining and Summarizing Customer Reviews." 
;       Proceedings of the ACM SIGKDD International Conference on Knowledge 
;       Discovery and Data Mining (KDD-2004), Aug 22-25, 2004, Seattle, 
;       Washington, USA, 
;   Bing Liu, Minqing Hu and Junsheng Cheng. "Opinion Observer: Analyzing 
;       and Comparing Opinions on the Web." Proceedings of the 14th 
;       International World Wide Web conference (WWW-2005), May 10-14, 
;       2005, Chiba, Japan.
;
; Notes: 
;    1. The appearance of an opinion word in a sentence does not necessarily  
;       mean that the sentence expres

#### Stop Words

In [41]:
with codecs.open("stop.txt", "r", encoding="ISO-8859-1") as s:
    stop = s.read()
    print(stop)

a
a's
able
about
above
according
accordingly
across
actually
after
afterwards
again
against
ain't
all
allow
allows
almost
alone
along
already
also
although
always
am
among
amongst
an
and
another
any
anybody
anyhow
anyone
anything
anyway
anyways
anywhere
apart
appear
appreciate
appropriate
are
aren't
around
as
aside
ask
asking
associated
at
available
away
awfully
b
be
became
because
become
becomes
becoming
been
before
beforehand
behind
being
believe
below
beside
besides
best
better
between
beyond
both
brief
but
by
c
c'mon
c's
came
can
can't
cannot
cant
cause
causes
certain
certainly
changes
clearly
co
com
come
comes
concerning
consequently
consider
considering
contain
containing
contains
corresponding
could
couldn't
course
currently
d
definitely
described
despite
did
didn't
different
do
does
doesn't
doing
don't
done
down
downwards
during
e
each
edu
eg
eight
either
else
elsewhere
enough
entirely
especially
et
etc
even
ever
every
everybody
everyone
everything
everywhere
ex
exactly
example

In [42]:
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to C:\Users\Punith
[nltk_data]     Gowda\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [43]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')

In [44]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')

In [45]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]

print(tweet_tokens[0])

#FollowFriday


In [51]:
no_pages = 2

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        #print(d)
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price = d.find('span', attrs={'class':'p13n-sc-price'})

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')

        if rating is not None:
            #print(rating.text)
            all1.append(rating.text)
        else:
            all1.append('-1')

        if users_rated is not None:
            #print(price.text)
            all1.append(users_rated.text)
        else:
            all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

In [52]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Book Name','Author','Rating','Customers_Rated', 'Price'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

In [53]:
amz = pd.read_csv("amazon_products.csv")

In [56]:
df1 = amz.copy()

In [57]:
df1.head()

,Book Name,Author,Rating,Customers_Rated,Price
0,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6 out of 5 stars,"18,035",₹308.00
1,The Psychology of Money,Morgan Housel,4.6 out of 5 stars,"13,000",₹277.00
2,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5 out of 5 stars,"29,410",₹399.00
3,My First Book of Pencil Control : Practice Pat...,Wonder House Books,4.4 out of 5 stars,"7,668",₹89.00
4,Atomic Habits: The life-changing million copy ...,James Clear,4.6 out of 5 stars,"22,694",₹428.00


In [58]:
df1.shape

(100, 5)

In [59]:
df1['Rating'] = df1['Rating'].apply(lambda x: x.split()[0])

In [60]:
df1['Rating'] = pd.to_numeric(df1['Rating'])

In [61]:
df1["Price"] = df1["Price"].str.replace('₹', '')

In [62]:
df1["Price"] = df1["Price"].str.replace(',', '')

In [64]:
df1['Price'] = df1['Price'].apply(lambda x: x.split('.')[0])

In [65]:
df1['Price'] = df1['Price'].astype(int)

In [66]:
df1["Customers_Rated"] = df1["Customers_Rated"].str.replace(',', '')

In [68]:
df1['Customers_Rated'] = pd.to_numeric(df1['Customers_Rated'], errors='ignore')

In [69]:
df1.head()

,Book Name,Author,Rating,Customers_Rated,Price
0,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6,18035,308
1,The Psychology of Money,Morgan Housel,4.6,13000,277
2,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5,29410,399
3,My First Book of Pencil Control : Practice Pat...,Wonder House Books,4.4,7668,89
4,Atomic Habits: The life-changing million copy ...,James Clear,4.6,22694,428


In [70]:
df1.dtypes

Book Name           object
Author              object
Rating             float64
Customers_Rated      int64
Price                int32
dtype: object

In [71]:
df1.replace(str(0), np.nan, inplace=True)
df1.replace(0, np.nan, inplace=True)

In [72]:
count_nan = len(df) - df1.count()

In [73]:
count_nan

Book Name          0
Author             0
Rating             0
Customers_Rated    0
Price              0
dtype: int64

In [74]:
df1 = df1.dropna()

In [75]:
data1 = df1.sort_values(["Price"], axis=0, ascending=False)[:15]

In [76]:
data1

,Book Name,Author,Rating,Customers_Rated,Price
54,My First Complete Learning Library: Boxset of ...,Wonder House Books,4.6,5674,790
60,How to Prepare for QUANTITATIVE APTITUDE for C...,Arun Sharma,4.5,83,740
51,"NCERT textbooks physics, chemistry and biology...",NCERT,4.1,1124,675
94,How to Prepare for Verbal Ability and Reading ...,Arun Sharma,4.5,1084,649
16,Indian Polity - For Civil Services and Other S...,M. Laxmikanth,4.6,11838,620
19,Objective NCERT at your FINGERTIPS for NEET-AI...,MTG Editorial Board,4.6,2654,556
29,A Modern Approach to Verbal & Non-Verbal Reaso...,R.S. Aggarwal,4.4,6422,545
78,All set to Read-A Phonic Reader-Level 1- PHONI...,Om Books Editorial Team,4.6,660,500
42,Best of Children’s Classics (Set of 5 Books): ...,Antoine de Saint-Exupéry,4.4,1066,499
53,Objective NCERT at your Fingertips for NEET-AI...,MTG Editorial Board,4.5,2065,470


In [80]:
import math
from bokeh.io import curdoc
curdoc().clear()
from bokeh.io import push_notebook, show, output_notebook
from bokeh.transform import factor_cmap
from bokeh.models import Legend
from bokeh.palettes import Dark2_5 as palette
from bokeh.plotting import figure
import itertools
from bokeh.palettes import d3
output_notebook()

Loading BokehJS ...

In [88]:
p = figure(x_range=data1.iloc[:,1], plot_width=800, plot_height=550, title="Authors Highest Priced Book", toolbar_location=None, tools="")

p.vbar(x=data1.iloc[:,1], top=data1.iloc[:,4], width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/2

In [89]:
print(p)

Figure(id='1069', ...)


In [90]:
data2 = df1[df1['Customers_Rated'] > 1000]

In [91]:
data2 = data2.sort_values(['Rating'],axis=0, ascending=False)[:15]

In [92]:
data2

,Book Name,Author,Rating,Customers_Rated,Price
33,Sapiens: A Brief History of Humankind,Yuval Noah Harari,4.7,35545,370
59,"The Magic of the Lost Temple: Illustrated, eas...",Sudha Murty,4.7,3214,154
41,Think Like a Monk: The secret of how to harnes...,Jay Shetty,4.7,15671,300
14,Karma: A Yogi's Guide to Crafting Your Destiny...,Sadhguru,4.7,2375,190
0,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6,18035,308
16,Indian Polity - For Civil Services and Other S...,M. Laxmikanth,4.6,11838,620
38,Life's Amazing Secrets: How to Find Balance an...,Gaur Gopal Das,4.6,9196,150
1,The Psychology of Money,Morgan Housel,4.6,13000,277
54,My First Complete Learning Library: Boxset of ...,Wonder House Books,4.6,5674,790
63,Mathematics for Class 9 by R D Sharma (Examina...,R.D. Sharma,4.6,1770,376


In [96]:
p1 = figure(x_range=data1.iloc[:,0], plot_width=800, plot_height=600, title="Top Rated Books with more than 1000 Customers Rating", toolbar_location=None, tools="")

p1.vbar(x=data1.iloc[:,0], top=data1.iloc[:,2], width=0.9)

p1.xgrid.grid_line_color = None
p1.y_range.start = 0
p1.xaxis.major_label_orientation = math.pi/2

In [97]:
print(p1)

Figure(id='1166', ...)


In [98]:
show(p1)

In [99]:
p2 = figure(x_range=data1.iloc[:,1], plot_width=800, plot_height=600, title="Top Rated Books with more than 1000 Customers Rating", toolbar_location=None, tools="")

p2.vbar(x=data1.iloc[:,1], top=data1.iloc[:,2], width=0.9)

p2.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/2

In [101]:
print(p2)

Figure(id='1246', ...)


In [104]:
data4 = df1.sort_values(["Customers_Rated"], axis=0, ascending=False)[:20]

In [105]:
data4

,Book Name,Author,Rating,Customers_Rated,Price
7,The Alchemist,Paulo Coelho,4.6,59751,212
36,Think and Grow Rich,Napoleon Hill,4.5,52240,139
84,How to Win Friends and Influence People,Dale Carnegie,4.5,51860,129
74,The Subtle Art of Not Giving a F*ck: A Counter...,Mark Manson,4.5,50703,325
8,Rich Dad Poor Dad: What the Rich Teach Their K...,Robert T. Kiyosaki,4.6,44370,332
90,To Kill A Mockingbird: 50th Anniversary Editio...,Harper Lee,4.5,41652,284
62,The Power of your Subconscious Mind,Joseph Murphy,4.5,35767,149
28,The Power of Your Subconscious Mind,Joseph Murphy,4.5,35767,141
33,Sapiens: A Brief History of Humankind,Yuval Noah Harari,4.7,35545,370
25,Man's Search For Meaning: The classic tribute ...,Viktor E Frankl,4.5,33454,183
